#  Multiple Regression and Feature Engineering

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1529352599.log


This non-commercial license of GraphLab Create for academic use is assigned to nanlee_89@yahoo.com and will expire on December 07, 2018.


# Split data into training and testing.  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learn a multiple regression model

In [4]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', 
                                                  features = example_features, 
                                                  verbose = False,
                                                  validation_set = None)

### Explore the regression weights (coefficients)

In [5]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019265  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions


In [6]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.505878


# Compute RSS

In [7]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)

    # Then compute the residuals/errors
    residuals = outcome - predictions

    # Then square and add them up
    squared_residuals = residuals*residuals
    
    RSS = squared_residuals.sum()

    return(RSS)    

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

In [8]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [9]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [10]:
# create the remaining 3 features in both TEST and TRAIN data

train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

In [11]:
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

In [12]:
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat']+ test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

### Mean (arithmetic average) value of  4 new features on TEST data

In [13]:
print "Then mean value of bedroom_squared on TEST data: ", test_data['bedrooms_squared'].mean()
print "Then mean value of bed_bath_rooms on TEST data: ", test_data['bed_bath_rooms'].mean()
print "Then mean value of log_sqft_living on TEST data: ", test_data['log_sqft_living'].mean()
print "Then mean value of lat_plus_long on TEST data: ", test_data['lat_plus_long'].mean()

Then mean value of bedroom_squared on TEST data:  12.4466777016
Then mean value of bed_bath_rooms on TEST data:  7.50390163159
Then mean value of log_sqft_living on TEST data:  7.55027467965
Then mean value of lat_plus_long on TEST data:  -74.6533349722


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [14]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

### Learn the three models: 


In [15]:
model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features,
                                            verbose=False, validation_set = None)
model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features, 
                                            verbose=False, validation_set = None)
model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features, 
                                            verbose=False, validation_set = None)

### Examine/extract each model's coefficients:


In [16]:
model_1_weight_summary = model_1.get("coefficients")
print "Model 1 coefficients \n", model_1_weight_summary

Model 1 coefficients 
+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -56140675.7444 | 1649985.42028 |
| sqft_living |  None | 310.263325778  | 3.18882960408 |
|   bedrooms  |  None | -59577.1160682 | 2487.27977322 |
|  bathrooms  |  None | 13811.8405418  | 3593.54213297 |
|     lat     |  None | 629865.789485  | 13120.7100323 |
|     long    |  None | -214790.285186 | 13284.2851607 |
+-------------+-------+----------------+---------------+
[6 rows x 4 columns]



In [17]:
model_2_weight_summary = model_2.get("coefficients")
print "Model 2 coefficients \n", model_2_weight_summary

Model 2 coefficients 
+----------------+-------+----------------+---------------+
|      name      | index |     value      |     stderr    |
+----------------+-------+----------------+---------------+
|  (intercept)   |  None | -54410676.1152 | 1650405.16541 |
|  sqft_living   |  None | 304.449298057  | 3.20217535637 |
|    bedrooms    |  None | -116366.043231 | 4805.54966546 |
|   bathrooms    |  None | -77972.3305135 | 7565.05991091 |
|      lat       |  None | 625433.834953  | 13058.3530972 |
|      long      |  None | -203958.60296  | 13268.1283711 |
| bed_bath_rooms |  None | 26961.6249092  | 1956.36561555 |
+----------------+-------+----------------+---------------+
[7 rows x 4 columns]



In [18]:
model_3_weight_summary = model_3.get("coefficients")
print "Model 3 coefficients \n", model_3_weight_summary

Model 3 coefficients 
+------------------+-------+----------------+---------------+
|       name       | index |     value      |     stderr    |
+------------------+-------+----------------+---------------+
|   (intercept)    |  None | -52974974.0602 |  1615194.9439 |
|   sqft_living    |  None | 529.196420564  | 7.69913498511 |
|     bedrooms     |  None | 28948.5277313  | 9395.72889106 |
|    bathrooms     |  None |  65661.207231  | 10795.3380703 |
|       lat        |  None | 704762.148408  | 1292011141.66 |
|       long       |  None | -137780.01994  | 1292011141.57 |
|  bed_bath_rooms  |  None | -8478.36410518 | 2858.95391257 |
| bedrooms_squared |  None | -6072.38466067 | 1494.97042777 |
| log_sqft_living  |  None | -563467.784269 | 17567.8230814 |
|  lat_plus_long   |  None | -83217.1979248 | 1292011141.58 |
+------------------+-------+----------------+---------------+
[10 rows x 4 columns]



# Comparing multiple models

Now that we've learned three models and extracted the model weights we want to evaluate which model is best.

### Compute the RSS on TRAINING data for each of the three models and record the values


In [19]:
rss_model_1_train = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
print "RSS of model 1:", rss_model_1_train

rss_model_2_train = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
print "RSS of model 2:", rss_model_2_train

rss_model_3_train = get_residual_sum_of_squares(model_3, train_data, train_data['price'])
print "RSS of model 3:", rss_model_3_train

RSS of model 1: 9.71328233544e+14
RSS of model 2: 9.61592067856e+14
RSS of model 3: 9.05276314555e+14


### Compute the RSS on on TEST data for each of the three models.

In [20]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_model_1_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
print "RSS of model 1:", rss_model_1_test

rss_model_2_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
print "RSS of model 2:", rss_model_2_test

rss_model_3_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])
print "RSS of model 3:", rss_model_3_test


RSS of model 1: 2.26568089093e+14
RSS of model 2: 2.24368799994e+14
RSS of model 3: 2.51829318952e+14
